In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from lj_mmcmd.mclj import MCLJ
from lj_mmcmd.mdvvlj import MDvvlj
import matplotlib.pyplot as plt

In [ ]:
import pint
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

In [ ]:
input_crds = np.load("lj_mmcmd/data/input.npy")

In [ ]:
step = 0
lj_system = MCLJ()
lj_system.system_size = 50
lj_system.sigma = Q_(3.405, ureg.angstrom)
lj_system.epsilon = Q_(0.238, "kcal/mol")
lj_system.temperature = 298
lj_system.nparticles = 50

simulation_steps = 1000
initial_position = input_crds
last_energy = lj_system.calc_potential_energy(initial_position)
del initial_position

while step < simulation_steps:
    next_postion = lj_system.new_positions()
    this_energy = lj_system.calc_potential_energy(next_postion)
    lj_system.decision_maker(last_energy, this_energy)
    if lj_system.accept == True:
        last_energy = this_energy
        lj_system.trajectories.append(next_postion)
        lj_system.potential_energies.append(this_energy)
    else:
        pass
    step += 1     

In [ ]:
plt.figure(figsize=(7, 5), dpi=100)
plt.title("MC-LJ")
plt.ylabel("Potential Energy (kcal/mol)")
plt.xlabel("Simulation steps")
plt.scatter([i for i in range(len(lj_system.potential_energies))], lj_system.potential_energies)
plt.show()

In [ ]:
# run 1ns simulations
test = MDvvlj(input_crds, 50) 
test.run(400000)

In [ ]:
plt.figure(figsize=(7, 5), dpi=100)
plt.title("MD-LJ")
plt.ylabel("Potential Energy (kcal/mol)")
plt.xlabel("Simulation steps")
plt.scatter([i for i in range(len(test.potential_energies))], test.potential_energies)
plt.show()

In [ ]:
total_energy = np.array(test.potential_energies) + np.array(test.kinetic_energies)

In [ ]:
plt.figure(figsize=(7, 5), dpi=100)
plt.title("MD-LJ")
plt.ylabel("Energy (kcal/mol)")
plt.xlabel("Simulation steps")
plt.scatter([i for i in range(len(total_energy))], total_energy, label="total")
plt.scatter([i for i in range(len(test.potential_energies))], test.potential_energies, label="potential")
plt.scatter([i for i in range(len(test.kinetic_energies))], test.kinetic_energies, label="kinetic")
plt.legend()
plt.show()